API - Datos Red Eléctrica

In [9]:
import requests
#import pandas as pd
import dotenv as dotenv
import os


# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import pandas as pd
import re

In [10]:
cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}

In [11]:
anios = [2019,2020,2021]
# me hago la lista de años que busco

API - Demanda Eléctrica

In [91]:
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": "apidatos.ree.es"}
# creo mi destino
ruta = "C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_demanda_electrica"
os.makedirs(ruta, exist_ok=True)
# si existe bien, si no pues la crea
for i in anios: #itero en años
    for name, cod in cod_comunidades.items(): #itero en comunicades
        url_demanda_electrica = f'https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={cod}'

        response = requests.get(url_demanda_electrica, headers=headers)
        if response.status_code == 200:
            response_json= response.json()
            df_demanda_electrica= pd.DataFrame(response_json['included'][0]['attributes']['values']) #tomo el primero porque solo hay uno
            df_demanda_electrica['COD-CCAA'] = cod
            df_demanda_electrica.to_csv(os.path.join(ruta, f'{name}_{i}.csv'))
        else: 
            print(f'El error es {response.status_code} en {name} para el annio {i}')

In [72]:
response_json

{'data': {'type': 'Evolución de la demanda',
  'id': 'dem1',
  'attributes': {'title': 'Evolución de la demanda',
   'last-update': '2022-01-11T09:21:30.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-11-28T05:55:18'}}},
 'included': [{'type': 'Demanda',
   'id': '10339',
   'groupId': None,
   'attributes': {'title': 'Demanda',
    'description': None,
    'color': '#ffea00',
    'type': None,
    'magnitude': None,
    'composite': False,
    'last-update': '2022-11-10T23:30:07.000+01:00',
    'values': [{'value': 146298.76,
      'percentage': 1,
      'datetime': '2021-01-01T00:00:00.000+01:00'},
     {'value': 129503.338,
      'percentage': 1,
      'datetime': '2021-02-01T00:00:00.000+01:00'},
     {'value': 144652.408,
      'percentage': 1,
      'datetime': '2021-03-01T00:00:00.000+01:00'},
     {'value': 129506.033,
      'percentage': 1,
      'datetime': '2021-04-01T00:00:00.000+02:00'},
     {'value': 130651.022,
   

In [73]:
response_json['included'][0]

{'type': 'Demanda',
 'id': '10339',
 'groupId': None,
 'attributes': {'title': 'Demanda',
  'description': None,
  'color': '#ffea00',
  'type': None,
  'magnitude': None,
  'composite': False,
  'last-update': '2022-11-10T23:30:07.000+01:00',
  'values': [{'value': 146298.76,
    'percentage': 1,
    'datetime': '2021-01-01T00:00:00.000+01:00'},
   {'value': 129503.338,
    'percentage': 1,
    'datetime': '2021-02-01T00:00:00.000+01:00'},
   {'value': 144652.408,
    'percentage': 1,
    'datetime': '2021-03-01T00:00:00.000+01:00'},
   {'value': 129506.033,
    'percentage': 1,
    'datetime': '2021-04-01T00:00:00.000+02:00'},
   {'value': 130651.022,
    'percentage': 1,
    'datetime': '2021-05-01T00:00:00.000+02:00'},
   {'value': 128050.949,
    'percentage': 1,
    'datetime': '2021-06-01T00:00:00.000+02:00'},
   {'value': 140864.511,
    'percentage': 1,
    'datetime': '2021-07-01T00:00:00.000+02:00'},
   {'value': 130850.035,
    'percentage': 1,
    'datetime': '2021-08-01T0

In [62]:
len(response_json['included'])

1

Api - Generación

In [112]:
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": "apidatos.ree.es"}

ruta_g = "C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_generacion_estructura"
os.makedirs(ruta_g, exist_ok=True)

for i in anios:
    for name, cod in cod_comunidades.items():
        url_generacion_renovable = f'https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=month&geo_trunk=electric_system&geo_limit=ccaa&geo_ids={cod}'

        response_generacion = requests.get(url_generacion_renovable, headers=headers) # me dio problemas, veamos el porqué
        if response_generacion.status_code == 200:
            response_generacion_json = response_generacion.json()
            lista_df = []
            for j in range(len(response_generacion_json['included'])): #itero aquí porque son varios elementos 
                df_generacion_estructura = pd.DataFrame(response_generacion_json['included'][j]['attributes']['values'])
                df_generacion_estructura['type'] = response_generacion_json['included'][j]['type']
                df_generacion_estructura['COD-CCAA'] = cod
                lista_df.append(df_generacion_estructura)
            df_final = pd.concat(lista_df).to_csv(os.path.join(ruta_g, f'{name}_{i}.csv'))
        else:
            print(f"Error {response_generacion.status_code} en la {name} para el año {i}")

        #recuerda que te equivocaste primero, prueba con response_json
        # será que se sobreescribe o se apendea
        # trata de poner el codigo de cada comunidad

Error 502 en la Ceuta para el año 2019
Error 502 en la Ceuta para el año 2020
Error 502 en la Ceuta para el año 2021


In [113]:
response_generacion_json.keys()

dict_keys(['data', 'included'])

In [114]:
response_generacion_json['included'][2]

{'type': 'Solar fotovoltaica',
 'id': '1707',
 'groupId': '1',
 'attributes': {'title': 'Solar fotovoltaica',
  'description': None,
  'color': '#e48500',
  'type': None,
  'magnitude': None,
  'composite': False,
  'last-update': '2022-11-15T19:06:23.000+01:00',
  'values': [{'value': 5153.449,
    'percentage': 0.03487737464753944,
    'datetime': '2021-01-01T00:00:00.000+01:00'},
   {'value': 7717.789,
    'percentage': 0.07062128878664413,
    'datetime': '2021-02-01T00:00:00.000+01:00'},
   {'value': 13143.273,
    'percentage': 0.09542179470416046,
    'datetime': '2021-03-01T00:00:00.000+01:00'},
   {'value': 13292.921,
    'percentage': 0.12720812141742366,
    'datetime': '2021-04-01T00:00:00.000+02:00'},
   {'value': 16882.492,
    'percentage': 0.24386430544372265,
    'datetime': '2021-05-01T00:00:00.000+02:00'},
   {'value': 16963.17,
    'percentage': 0.25266835545486194,
    'datetime': '2021-06-01T00:00:00.000+02:00'},
   {'value': 18335.209,
    'percentage': 0.2095632

In [115]:
len(response_generacion_json['included'])

5

In [116]:
len(response_generacion_json['included'][1]['attributes']['values'])

12

In [117]:
response_generacion_json['included'][1]

{'type': 'Eólica',
 'id': '10333',
 'groupId': '1',
 'attributes': {'title': 'Eólica',
  'description': None,
  'color': '#6fb114',
  'type': None,
  'magnitude': None,
  'composite': False,
  'last-update': '2022-11-10T23:30:06.000+01:00',
  'values': [{'value': 126564.559,
    'percentage': 0.8565602456420176,
    'datetime': '2021-01-01T00:00:00.000+01:00'},
   {'value': 80023.713,
    'percentage': 0.7322534660577698,
    'datetime': '2021-02-01T00:00:00.000+01:00'},
   {'value': 106078.27,
    'percentage': 0.7701414177817432,
    'datetime': '2021-03-01T00:00:00.000+01:00'},
   {'value': 78024.301,
    'percentage': 0.7466624344730259,
    'datetime': '2021-04-01T00:00:00.000+02:00'},
   {'value': 39047.666,
    'percentage': 0.564035922438966,
    'datetime': '2021-05-01T00:00:00.000+02:00'},
   {'value': 38798.185,
    'percentage': 0.5779033988684599,
    'datetime': '2021-06-01T00:00:00.000+02:00'},
   {'value': 55639.61,
    'percentage': 0.635935918560497,
    'datetime': '

Concatenar csv para:
* Demanda eléctrica

In [182]:
# Vamos a definir las rutas de las carpetas de donde sacamos los archivos y donde queremos almacenar los resutlados después de toda la transformación
ruta_origen = 'C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_demanda_electrica'
df = pd.DataFrame() #crear df para concatenar
for archivo in (os.listdir(ruta_origen)):
    print(archivo)
    # ruta_archivo = os.path.join(ruta_origen, archivo)
    df_variante = pd.read_csv(f'C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_demanda_electrica\\{archivo}', index_col=0)
    print(df_variante)
    #creo uno dentro 
    #leer el csv{archivo}
    nombre_limpio = archivo.split('.csv') #split.csv
    print(nombre_limpio)
    nombre_junto_ano = nombre_limpio[0].split('_')
    nombre = nombre_junto_ano[0]
    fecha = nombre_junto_ano[1]
    df_variante['CCAA'] = nombre
    df_variante['Año'] = fecha
    df = pd.concat([df, df_variante])
    #saconombre
    #sacoaño
    #creo columnas nuevas, 
    #creo data frame

Andalucía_2019.csv
          value  percentage                       datetime  COD-CCAA
0   3668893.328           1  2019-01-01T00:00:00.000+01:00         4
1   3152278.091           1  2019-02-01T00:00:00.000+01:00         4
2   3216356.360           1  2019-03-01T00:00:00.000+01:00         4
3   3029639.973           1  2019-04-01T00:00:00.000+02:00         4
4   3154123.467           1  2019-05-01T00:00:00.000+02:00         4
5   3181997.313           1  2019-06-01T00:00:00.000+02:00         4
6   3676269.543           1  2019-07-01T00:00:00.000+02:00         4
7   3786647.665           1  2019-08-01T00:00:00.000+02:00         4
8   3261926.084           1  2019-09-01T00:00:00.000+02:00         4
9   3143205.713           1  2019-10-01T00:00:00.000+02:00         4
10  3215694.404           1  2019-11-01T00:00:00.000+01:00         4
11  3383670.228           1  2019-12-01T00:00:00.000+01:00         4
['Andalucía_2019', '']
Andalucía_2020.csv
          value  percentage               

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 11
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       684 non-null    float64
 1   percentage  684 non-null    int64  
 2   datetime    684 non-null    object 
 3   COD-CCAA    684 non-null    int64  
 4   CCAA        684 non-null    object 
 5   Año         684 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 37.4+ KB


In [184]:
df.head()

,value,percentage,datetime,COD-CCAA,CCAA,Año
0,3668893.328,1,2019-01-01T00:00:00.000+01:00,4,Andalucía,2019
1,3152278.091,1,2019-02-01T00:00:00.000+01:00,4,Andalucía,2019
2,3216356.360,1,2019-03-01T00:00:00.000+01:00,4,Andalucía,2019
3,3029639.973,1,2019-04-01T00:00:00.000+02:00,4,Andalucía,2019
4,3154123.467,1,2019-05-01T00:00:00.000+02:00,4,Andalucía,2019


In [185]:
# Convierte la columna a formato datetime
df['datetime'] = pd.to_datetime(df['datetime'], utc= False)


C:\Users\DELL\AppData\Local\Temp\ipykernel_9564\404347714.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['datetime'] = pd.to_datetime(df['datetime'], utc= False)


In [216]:
df['datetime'] = df['datetime'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))

df['Mes'] = df['datetime'].apply(lambda x: pd.to_datetime(x).month)

In [228]:
df.head()

,value,percentage,COD-CCAA,CCAA,Año,Mes
0,3668893.328,1,4,Andalucía,2019,1
1,3152278.091,1,4,Andalucía,2019,2
2,3216356.360,1,4,Andalucía,2019,3
3,3029639.973,1,4,Andalucía,2019,4
4,3154123.467,1,4,Andalucía,2019,5


In [ ]:
df = df.drop(columns='datetime')

In [218]:
# Vamos a definir las rutas de las carpetas de donde sacamos los archivos y donde queremos almacenar los resutlados después de toda la transformación
ruta_origen = 'C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_generacion_estructura'
df_generacion = pd.DataFrame() #crear df para concatenar
for archivo in (os.listdir(ruta_origen)):
    ruta_archivo = os.path.join(ruta_origen, archivo)
    df_variante_gen = pd.read_csv(ruta_archivo, index_col=0)
    #creo uno dentro 
    #leer el csv{archivo}
    nombre_limpio_gen = archivo.split('.csv') #split.csv
    # print(nombre_limpio_gen)
    nombre_junto_ano_gen = nombre_limpio_gen[0].split('_')
    nombre_gen = nombre_junto_ano_gen[0]
    # print(nombre_gen)
    fecha_gen = nombre_junto_ano_gen[1]
    # print(fecha_gen)
    df_variante_gen['CCAA'] = nombre_gen
    df_variante_gen['Año'] = fecha_gen
    df_generacion = pd.concat([df_generacion, df_variante_gen])

In [219]:
df_generacion[df_generacion['CCAA'] == 'Andalucía']

,value,percentage,datetime,type,COD-CCAA,CCAA,Año
0,25288.605,0.029517,2019-01-01T00:00:00.000+01:00,Hidráulica,4,Andalucía,2019
1,33933.350,0.030779,2019-02-01T00:00:00.000+01:00,Hidráulica,4,Andalucía,2019
2,35913.000,0.029645,2019-03-01T00:00:00.000+01:00,Hidráulica,4,Andalucía,2019
3,44050.848,0.041340,2019-04-01T00:00:00.000+02:00,Hidráulica,4,Andalucía,2019
4,88535.452,0.068497,2019-05-01T00:00:00.000+02:00,Hidráulica,4,Andalucía,2019
...,...,...,...,...,...,...,...
7,1359426.900,1.000000,2021-08-01T00:00:00.000+02:00,Generación renovable,4,Andalucía,2021
8,1200892.817,1.000000,2021-09-01T00:00:00.000+02:00,Generación renovable,4,Andalucía,2021
9,1345846.061,1.000000,2021-10-01T00:00:00.000+02:00,Generación renovable,4,Andalucía,2021
10,1101043.737,1.000000,2021-11-01T00:00:00.000+01:00,Generación renovable,4,Andalucía,2021


value a mwh

In [220]:
df_generacion['Año'].unique()

array(['2019', '2020', '2021'], dtype=object)

In [221]:
df_generacion['datetime'] = df_generacion['datetime'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))


In [222]:
df_generacion['Mes'] = df_generacion['datetime'].apply(lambda x: pd.to_datetime(x).month)

In [223]:
# df_generacion['año'] = df_generacion['datetime'].apply(lambda x: pd.to_datetime(x).year)

In [224]:
df_generacion.head()

,value,percentage,datetime,type,COD-CCAA,CCAA,Año,Mes
0,25288.605,0.029517,2019-01,Hidráulica,4,Andalucía,2019,1
1,33933.350,0.030779,2019-02,Hidráulica,4,Andalucía,2019,2
2,35913.000,0.029645,2019-03,Hidráulica,4,Andalucía,2019,3
3,44050.848,0.041340,2019-04,Hidráulica,4,Andalucía,2019,4
4,88535.452,0.068497,2019-05,Hidráulica,4,Andalucía,2019,5


In [225]:
df_generacion = df_generacion.drop(columns='datetime')

In [232]:
df_generacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3557 entries, 0 to 11
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       3557 non-null   float64
 1   percentage  3557 non-null   float64
 2   type        3557 non-null   object 
 3   COD-CCAA    3557 non-null   int64  
 4   CCAA        3557 non-null   object 
 5   Año         3557 non-null   object 
 6   Mes         3557 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 222.3+ KB


In [243]:
df_generacion[(df_generacion['CCAA'] == 'Andalucía') & (df_generacion['Año'] == '2019') & (df_generacion['Mes'] == 12)]

,value,percentage,type,COD-CCAA,CCAA,Año,Mes
11,33252.448,0.033001,Hidráulica,4,Andalucía,2019,12
11,684672.055,0.679492,Eólica,4,Andalucía,2019,12
11,120605.521,0.119693,Solar fotovoltaica,4,Andalucía,2019,12
11,38142.147,0.037854,Solar térmica,4,Andalucía,2019,12
11,130950.924,0.129960,Otras renovables,4,Andalucía,2019,12
11,1007623.095,1.000000,Generación renovable,4,Andalucía,2019,12
